In [41]:
## To check GPU
!nvidia-smi

In [42]:
import numpy as np
import torch

In [43]:
input_data = np.array([
    
                      [32,55,77],
                      [31,75,57],
                      [52,55,77],
                      [22,100,87],
                      [62,80,77]
                      
                      ], dtype='float32')

In [44]:
input_data

In [45]:
target_data = np.array([
    
                       [32,66],
                       [52,76],
                       [102,155],
                       [32,26],
                       [99,100]
                       
                       ])

In [46]:
target_data

In [47]:
## Converting numpy array into tensor so that we can make use of GPU
input_tensor = torch.from_numpy(input_data)
target_tensor = torch.from_numpy(target_data)

print(input_tensor)
print(target_tensor)

In [48]:
## Initialize weigts and biases

w = torch.rand(2,3, requires_grad = True)
b = torch.rand(2, requires_grad = True)

print(w)
print(b)

In [49]:
def model(x):
    return x @ w.t() + b

In [50]:
## Prediction
preds = model(input_tensor)
print(preds)

In [51]:
## Actual
target_tensor

In [52]:
## Loss Function

def MSE(t1, t2):
    diff = t1 - t2
    return torch.sum(diff * diff) / diff.numel()

In [53]:
## Compute Loss

loss = MSE(preds, target_tensor)
print(loss)

In [54]:
## RMSE
import math
rmse = math.sqrt(loss)
rmse

To optimize weights of parameters, we need to compute the **derivatives** of our loss function with respect to parameters, under some fixed values of x and y. To compute those derivatives, we call loss.backward(), and then retrieve the values from w.grad and b.grad:


In [55]:
loss.backward()

In [56]:
print(w)
print(w.grad)

In [57]:
with torch.no_grad():
    w -= w.grad * 1e-5
    b -= b.grad * 1e-5

In [58]:
print(w)

In [59]:
pred1 = model(input_tensor)

loss1 = MSE(pred1,target_tensor)
print(f"Previous Loss is: {loss}")
print(f"Current Loss is: {loss1}")

In [60]:
## Training Loop

n = 100
for i in range(n):
    prediction = model(input_tensor)
    loss_ = MSE(prediction,target_tensor)
    # for calculating gradient
    loss_.backward()
    
    with torch.no_grad():
        w -= w.grad * 1e-5  # w(new) = w(old) - learning_rate*(dy/dx)
        b -= b.grad * 1e-5
        w.grad.zero_()
        b.grad.zero_()
        
    print(f"Epochs: {i}/{n} ---- Loss: {loss_}")

In [61]:
pred2 = model(input_tensor)
pred2

In [62]:
target_tensor

#### Linear Regression Using PyTorch InBuilt Functions

In [63]:
import torch.nn as nn

In [64]:
# input (temp, rainfall, humidity)

inputs = np.array([[73, 67, 43],
            [91, 88, 64],
            [87, 134, 58],
            [102, 43, 37],
            [69, 96, 70],
            [74, 66, 43],
            [91, 87, 65],
            [88, 134, 59],
            [101, 44, 37],
            [68, 96, 71],
            [73, 66, 44],
            [92, 87, 64],
            [87, 135, 57],
            [103, 43, 36],
            [68, 97, 70]], dtype='float32')

targets = np.array([[56,70],
                   [81, 101],
                   [119, 133],
                   [22, 37],
                   [103, 119],
                   [57, 69],
                   [80, 102],
                   [118, 132],
                   [21, 38],
                   [104, 118],
                   [57, 69],
                   [82, 100],
                   [118, 134],
                   [20, 38],
                   [102, 120]], dtype='float32')

inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)

In [65]:
inputs

In [66]:
# Data Loader
from torch .utils.data import TensorDataset

In [67]:
train_ds = TensorDataset(inputs, targets)
train_ds[0:3]

In [68]:
from torch.utils.data import DataLoader

batch_size = 5
train_data = DataLoader(train_ds, batch_size, shuffle=True)

In [69]:
for xb, yb in train_data:
    print(f"Input Data: \n{xb}\n")
    print(f"Target Data: \n{yb}")
    break

In [70]:
## Neural Networl Linear Layer

## Define Model

model = nn.Linear(3,2)
print(model.weight)
print(model.bias)

In [71]:
list(model.parameters())

In [72]:
## Passing Complete Data
preds = model(inputs)
preds

In [73]:
import torch.nn.functional as F
loss_fn = F.mse_loss

In [74]:
loss = loss_fn(preds, targets)
loss

In [75]:
## Optimizer

optim = torch.optim.SGD(model.parameters(), lr = 1e-5)

In [76]:
## Training Loop

def fit(num_epochs, model, loss_fn, opt,train_data):
    
    for epoch in range(num_epochs):
        for xb, yb in train_data:
            pred = model(xb)
            loss = loss_fn(pred, yb)
            loss.backward()
            optim.step()
            optim.zero_grad()
            
        if (epoch+1) % 10 == 0:
            print("Epoch [{}/{}], Loss: {:.4f}".format(epoch+1, num_epochs, loss))

In [77]:
## Call our fit function

fit(100, model, loss_fn, optim, train_data)

In [78]:
preds = model(inputs)
preds

In [79]:
targets

In [80]:
model(torch.tensor([[75,66,44.]]))